# Homework 2: SQL in Python (Data and Databases 2019)

The next four homework assignments will be in this iPython notebook format. Read the instructions as you go, and type in the necessary code--then run the cells to see if you got it right. Don't worry if you are new to python for this assignment--the only "code" you will be writing are SQL statements that you will paste into each myquery variable.


## The Challenges

For this homework we will be using the tables we imported from the [UN data site](http://data.un.org/Explorer.aspx). You should already have `wind` and `solar` installed. As you will see, this homework asks that you find one more table of your choice from the UN's Energy statistics an import it into the the same database. [I have posted the instructions here](http://floatingmedia.com/columbia/csv_to_sql.html)

These are relatively simple CSV files--the only really important columns are `country`, `year` and `usage`. But the tricky part is that most countries have multiple entries: one for every year the UN has been gathering data on that country. So most of these queries are going to ask you to get electrical usage results based on country and year. (Major hint: JOINs are going to need to be based on country AND year.)

In the questions below, we focus on exploratory queries--with an emphasis on aggregates and joins. Read the questions carefully, and then write SQL statements that will get you the correct results. (You will want to explore tables and test out queries directly in the shell before you paste them into Python--the error handling for SQL in Python can be confusing, and there should be no errors in the prewritten Python parts of this document. Also, sometimes an error in Python will break your connection with the database--see conn.rollback() below for that solution.


First things first: Connect to database! 
(Make sure the cell is selected, then go to the cell menubar and choose "Run Cells")

In [1]:
import pg8000
conn = pg8000.connect(database="unenergy2019",user="minami")
#conn = pg8000.connect(database="undata")
print(type(conn))

<class 'pg8000.core.Connection'>


If the connection isn't working make sure that you have typed in the name of the database correctly, and your username as well. 

If you get an error when running one of the SQL cells below, run this conn.rollback() cell to restart your connection.

In [2]:
conn.rollback()

## Question 1: Simple(ish) Query 

Create a statement that searches for the top 15 countries for solar energy usage in 2014. So figure out first what table you are searching and what columns you need (you can figure out the columns by looking at the sample results below). Then you need to test by year so you only get results from 2014, and order it correctly.

```
Germany 36056.0 mil
United States 25764.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 5913.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2883.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil
```

In [3]:
cursor = conn.cursor()
myquery = '''
SELECT country, usage FROM solar 
WHERE year = 2014 
ORDER BY usage DESC NULLS LAST 
LIMIT 15
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1], "mil")

Germany 36056.0 mil
United States 25764.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 5913.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2883.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil


## Question 2: aggregate query

Using avg(), get the average solar energy usage for each country, and it display only the countries with an average that is above 500 (million). (Remember for aggregates you need to use HAVING to test values, instead of WHERE). Make sure it is properly ordered like the results below, and wrap your avg column in round() to get round numbers:

```
20251.0 China
8482.0 Germany
7018.0 India
6254.0 United States
5604.0 Japan
4508.0 Italy
3423.0 Spain
1651.0 South Africa
1439.0 France
1431.0 United Kingdom
1369.0 Greece
1237.0 Czechia
1212.0 Belgium
991.0 Australia
884.0 Thailand
789.0 Bulgaria
731.0 Chile
649.0 Honduras
633.0 Korea, Republic of
585.0 Romania
581.0 Israel
```

In [4]:
cursor = conn.cursor()
myquery = '''
SELECT country, ROUND(AVG(usage)) as avg_usage FROM solar
GROUP BY country
HAVING AVG(usage) > 500
ORDER BY AVG(usage) DESC
'''

cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1])

China 20251.0
Germany 8482.0
India 7018.0
United States 6254.0
Japan 5604.0
Italy 4508.0
Spain 3423.0
South Africa 1651.0
France 1439.0
United Kingdom 1431.0
Greece 1369.0
Czechia 1237.0
Belgium 1212.0
Australia 991.0
Thailand 884.0
Bulgaria 789.0
Chile 731.0
Honduras 649.0
Korea, Republic of 633.0
Romania 585.0
Israel 581.0


## Question 3: aggregate query 2

Get a list of countries whose Solar Energy data first started being collected in 2012. This is tricky to figure out, but the query should be pretty simple. The main thing you need to figure out is which aggregate function will find the earliest date, and then you need to test if year is 2012. And make it alphabetical by country.


```
2012 American Samoa
2012 Botswana
2012 Gabon
2012 Lao People's Dem. Rep.
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 St. Kitts-Nevis
2012 United Arab Emirates
2012 Venezuela (Bolivar. Rep.)
```


In [5]:
cursor = conn.cursor()
myquery = '''
SELECT country, MIN(year) FROM solar
GROUP BY country
HAVING MIN(year) = 2012
ORDER BY country
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1])

American Samoa 2012
Botswana 2012
Gabon 2012
Lao People's Dem. Rep. 2012
Lithuania 2012
Marshall Islands 2012
Mauritius 2012
Poland 2012
South Sudan 2012
St. Helena and Depend. 2012
St. Kitts-Nevis 2012
United Arab Emirates 2012
Venezuela (Bolivar. Rep.) 2012


## Question 4: JOIN

Get the usage of solar AND wind power for Germany for each year in the tables. There should be four columns total, and you will be joining on two criteria.

(Please note that in the results below I am using Python to add in the strings "solar:" and "wind:")

```
Germany 1991 	wind: 215.0 	solar 1.0
Germany 1992 	wind: 291.0 	solar 4.0
Germany 1993 	wind: 674.0 	solar 3.0
Germany 1994 	wind: 1428.0 	solar 7.0
Germany 1995 	wind: 1712.0 	solar 7.0
Germany 1996 	wind: 2078.0 	solar 12.0
Germany 1997 	wind: 3034.0 	solar 18.0
Germany 1998 	wind: 4593.0 	solar 35.0
Germany 1999 	wind: 5528.0 	solar 30.0
Germany 2000 	wind: 9352.0 	solar 60.0
Germany 2001 	wind: 10456.0 	solar 116.0
Germany 2002 	wind: 15856.0 	solar 188.0
Germany 2003 	wind: 18713.0 	solar 313.0
Germany 2004 	wind: 25509.0 	solar 557.0
Germany 2005 	wind: 27229.0 	solar 1282.0
Germany 2006 	wind: 30710.0 	solar 2220.0
Germany 2007 	wind: 39713.0 	solar 3075.0
Germany 2008 	wind: 40574.0 	solar 4420.0
Germany 2009 	wind: 38647.0 	solar 6584.0
Germany 2010 	wind: 37793.0 	solar 11729.0
Germany 2011 	wind: 48883.0 	solar 19599.0
Germany 2012 	wind: 50670.0 	solar 26380.0
Germany 2013 	wind: 51708.0 	solar 31010.0
Germany 2014 	wind: 57357.0 	solar 36056.0
Germany 2015 	wind: 79206.0 	solar 38726.0
Germany 2016 	wind: 78598.0 	solar 38098.0
```

Hint: after the join you need to test that the country is Germany, then order by what?

In [6]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, solar.year, wind.usage AS wind_usage, solar.usage AS solar_usage
FROM solar
JOIN wind ON solar.country = wind.country AND solar.year = wind.year
WHERE solar.country = 'Germany'
ORDER BY year
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"\twind:",row[2],"\tsolar",row[3])

Germany 1991 	wind: 215.0 	solar 1.0
Germany 1992 	wind: 291.0 	solar 4.0
Germany 1993 	wind: 674.0 	solar 3.0
Germany 1994 	wind: 1428.0 	solar 7.0
Germany 1995 	wind: 1712.0 	solar 7.0
Germany 1996 	wind: 2078.0 	solar 12.0
Germany 1997 	wind: 3034.0 	solar 18.0
Germany 1998 	wind: 4593.0 	solar 35.0
Germany 1999 	wind: 5528.0 	solar 30.0
Germany 2000 	wind: 9352.0 	solar 60.0
Germany 2001 	wind: 10456.0 	solar 116.0
Germany 2002 	wind: 15856.0 	solar 188.0
Germany 2003 	wind: 18713.0 	solar 313.0
Germany 2004 	wind: 25509.0 	solar 557.0
Germany 2005 	wind: 27229.0 	solar 1282.0
Germany 2006 	wind: 30710.0 	solar 2220.0
Germany 2007 	wind: 39713.0 	solar 3075.0
Germany 2008 	wind: 40574.0 	solar 4420.0
Germany 2009 	wind: 38647.0 	solar 6584.0
Germany 2010 	wind: 37793.0 	solar 11729.0
Germany 2011 	wind: 48883.0 	solar 19599.0
Germany 2012 	wind: 50670.0 	solar 26380.0
Germany 2013 	wind: 51708.0 	solar 31010.0
Germany 2014 	wind: 57357.0 	solar 36056.0
Germany 2015 	wind: 79206.0 	

## Question 5: join with aggregates

Get the average wind usage and solar usage for each country. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 125453.67 	solar: 20251.33
Germany wind: 26174.12 	solar: 8481.92
India wind: 31553.75 	solar: 7017.50
United States wind: 52771.48 	solar: 6254.37
Japan wind: 1906.48 	solar: 5604.00
Italy wind: 4556.04 	solar: 4507.56
Spain wind: 20005.67 	solar: 3423.07
South Africa wind: 1827.50 	solar: 1651.25
France wind: 5877.04 	solar: 1439.13
United Kingdom wind: 12232.22 	solar: 1430.72
```

In [7]:
cursor = conn.cursor()
myquery = '''
SELECT 
solar.country, AVG(wind.usage) avg_wind_usage, AVG(solar.usage) avg_solar_usage 
FROM solar
JOIN wind ON solar.country = wind.country
GROUP BY solar.country
ORDER BY AVG(solar.usage) DESC
LIMIT 10
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 107072.56 	solar: 20251.33
Germany wind: 26174.12 	solar: 8481.92
India wind: 10165.74 	solar: 7017.50
United States wind: 52771.48 	solar: 6254.37
Japan wind: 1906.48 	solar: 5604.00
Italy wind: 4556.04 	solar: 4507.56
Spain wind: 20005.67 	solar: 3423.07
South Africa wind: 1059.71 	solar: 1651.25
France wind: 5632.25 	solar: 1439.13
United Kingdom wind: 8267.52 	solar: 1430.72


## Question 6: same thing but slightly more complicated

Get the average wind usage and solar usage for each country, FROM 2010 TO THE PRESENT. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 180028.00 	solar: 30278.75
Germany wind: 57745.00 	solar: 28799.71
United States wind: 161998.86 	solar: 21129.57
Japan wind: 5059.00 	solar: 19511.57
Italy wind: 13571.00 	solar: 17215.00
Spain wind: 48935.86 	solar: 11833.14
India wind: 31553.75 	solar: 7017.50
France wind: 16120.14 	solar: 4683.43
United Kingdom wind: 26305.14 	solar: 3667.00
Australia wind: 8569.29 	solar: 3260.29
```

In [8]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, AVG(wind.usage), AVG(solar.usage) FROM solar
JOIN wind ON solar.country = wind.country
WHERE solar.year > 2009 AND wind.year > 2009
GROUP BY solar.country
ORDER BY AVG(solar.usage) DESC
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 133006.14 	solar: 30278.75
Germany wind: 57745.00 	solar: 28799.71
United States wind: 161998.86 	solar: 21129.57
Japan wind: 5059.00 	solar: 19511.57
Italy wind: 13571.00 	solar: 17215.00
Spain wind: 48935.86 	solar: 11833.14
India wind: 28095.43 	solar: 7017.50
France wind: 16120.14 	solar: 4683.43
United Kingdom wind: 26305.14 	solar: 3667.00
Australia wind: 8569.29 	solar: 3260.29
Greece wind: 3924.86 	solar: 2533.14
Korea, Republic of wind: 1131.00 	solar: 2293.14
Belgium wind: 3663.57 	solar: 2220.43
Czechia wind: 453.71 	solar: 1928.29
South Africa wind: 1059.71 	solar: 1651.25
Canada wind: 18276.29 	solar: 1607.57
Thailand wind: 204.71 	solar: 1261.57
Bulgaria wind: 1192.14 	solar: 901.71
Romania wind: 4101.14 	solar: 835.29
Chile wind: 1091.43 	solar: 731.33
Israel wind: 6.67 	solar: 660.57
Honduras wind: 400.33 	solar: 649.00
Switzerland wind: 86.43 	solar: 622.29
Netherlands wind: 5888.57 	solar: 620.00
Other Asia wind: 1432.56 	solar: 615.20
Austria wind: 3362.2

## Question 7: Building a basic timeseries

Step one: get a list of the first year of data and the most recent year of data for each country from the solar table. Limit the results to the first 15 with the oldest year and then in alphabetical order.

```
Italy 1990 -- 2016
Japan 1990 -- 2016
Korea, Republic of 1990 -- 2016
Mexico 1990 -- 2016
Portugal 1990 -- 2016
Spain 1990 -- 2016
Switzerland 1990 -- 2016
United States 1990 -- 2016
Finland 1991 -- 2016
Germany 1991 -- 2016
Netherlands 1991 -- 2016
Canada 1992 -- 2016
Australia 1993 -- 2016
Austria 1993 -- 2016
Sweden 1993 -- 2016
```

In [9]:
cursor = conn.cursor()
myquery = '''
SELECT country, MIN(year), MAX(year) FROM solar
GROUP BY country
ORDER BY MIN(year), country
LIMIT 15
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2])

Italy 1990 -- 2016
Japan 1990 -- 2016
Korea, Republic of 1990 -- 2016
Mexico 1990 -- 2016
Portugal 1990 -- 2016
Spain 1990 -- 2016
Switzerland 1990 -- 2016
United States 1990 -- 2016
Finland 1991 -- 2016
Germany 1991 -- 2016
Netherlands 1991 -- 2016
Canada 1992 -- 2016
Australia 1993 -- 2016
Austria 1993 -- 2016
Sweden 1993 -- 2016


## Question 8: Building a basic timeseries

Step two: Take that previous query, (order, limit and everything) and turn that into a subquery, and join the solar table on it twice, first to get the usage from the earliest date, next get the usage from the most recent date. Order it by the most usage from the most recent year.

Note: to join at the same table twice, you need to alias the table (solar as sl1, solar as sl2). This is getting very challenging, so get as far as you can.

```
Japan 1990 -- 2016 usage change:  67.0 -- 50952.0
United States 1990 -- 2016 usage change:  666.0 -- 50334.0
Germany 1991 -- 2016 usage change:  1.0 -- 38098.0
Italy 1990 -- 2016 usage change:  4.0 -- 22104.0
Spain 1990 -- 2016 usage change:  6.0 -- 13649.0
Australia 1993 -- 2016 usage change:  11.0 -- 6209.0
Korea, Republic of 1990 -- 2016 usage change:  1.0 -- 5123.0
Canada 1992 -- 2016 usage change:  2.0 -- 3031.0
Netherlands 1991 -- 2016 usage change:  1.0 -- 1560.0
Switzerland 1990 -- 2016 usage change:  1.0 -- 1333.0
Austria 1993 -- 2016 usage change:  1.0 -- 1096.0
Portugal 1990 -- 2016 usage change:  1.0 -- 822.0
Mexico 1990 -- 2016 usage change:  1.0 -- 252.0
Sweden 1993 -- 2016 usage change:  1.0 -- 143.0
Finland 1991 -- 2016 usage change:  1.0 -- 17.0
```

In [16]:
# conn.rollback()

cursor = conn.cursor()
myquery = '''
WITH mm_tbl AS (
SELECT country, MIN(year) as min_yr, MAX(year) as max_yr
FROM solar
GROUP BY country
ORDER BY MIN(year), country
LIMIT 15
)
SELECT mm_tbl.country, mm_tbl.min_yr, mm_tbl.max_yr, solar1.usage min_usage, solar2.usage max_usage
FROM mm_tbl
JOIN solar as solar1 ON solar1.year = mm_tbl.min_yr AND solar1.country = mm_tbl.country
JOIN solar as solar2 ON solar2.year = mm_tbl.max_yr AND solar2.country = mm_tbl.country
ORDER BY max_usage DESC
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2],"usage change: ", row[3],"--",row[4])

Japan 1990 -- 2016 usage change:  67.0 -- 50952.0
United States 1990 -- 2016 usage change:  666.0 -- 50334.0
Germany 1991 -- 2016 usage change:  1.0 -- 38098.0
Italy 1990 -- 2016 usage change:  4.0 -- 22104.0
Spain 1990 -- 2016 usage change:  6.0 -- 13649.0
Australia 1993 -- 2016 usage change:  11.0 -- 6209.0
Korea, Republic of 1990 -- 2016 usage change:  1.0 -- 5123.0
Canada 1992 -- 2016 usage change:  2.0 -- 3031.0
Netherlands 1991 -- 2016 usage change:  1.0 -- 1560.0
Switzerland 1990 -- 2016 usage change:  1.0 -- 1333.0
Austria 1993 -- 2016 usage change:  1.0 -- 1096.0
Portugal 1990 -- 2016 usage change:  1.0 -- 822.0
Mexico 1990 -- 2016 usage change:  1.0 -- 252.0
Sweden 1993 -- 2016 usage change:  1.0 -- 143.0
Finland 1991 -- 2016 usage change:  1.0 -- 17.0


## Question 9: join with aggregates, new data

Your final task: go back to the [UN data site](http://data.un.org/Explorer.aspx) and download another table of your choice from the energy statistics. [Full instructions here.](http://floatingmedia.com/columbia/csv_to_sql.html) Choose a table that has similar columns to the wind and energy tables, and input it into your database.  Make sure you test that everything is working in the shell before going back to Python.

Now, write a query that joins the third table to the query you made in Question 5. Get the average use for each Energy type: Solar, Wind, and the table you chose. Order it by the energy type you chose (that column should come first after the country). And display the top 10 countries by the average of the energy type that you added, along with the averages for solar and wind.


In [17]:
cursor = conn.cursor()
myquery = '''
SELECT 
solar.country,
AVG(solar.usage) avg_solar_usage,
AVG(wind.usage) avg_wind_usage,
AVG(natural_gas.usage) avg_natural_gas_usage
FROM solar
JOIN wind ON solar.country = wind.country
JOIN natural_gas ON solar.country = natural_gas.country
GROUP BY solar.country
ORDER BY AVG(solar.usage) DESC
LIMIT 10
'''

cursor.execute(myquery)
for row in cursor:
    print(row[0],"%.2f" % row[1],"%.2f" % row[2],"%.2f" % row[3])

China 20251.33 107072.56 1714842.04
Germany 8481.92 26174.12 553695.97
India 7017.50 10165.74 1014142.39
United States 6254.37 52771.48 6310825.60
Japan 5604.00 1906.48 501830.62
Italy 4507.56 4556.04 484336.80
Spain 3423.07 20005.67 136683.47
South Africa 1651.25 1059.71 272890.71
France 1439.13 5632.25 251115.76
United Kingdom 1430.72 8267.52 963477.50


Great work! You're done.